In [274]:
import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session
import json

### Step 1: Get token

In [275]:
client_id = '2bd5c3ac-5128-4527-b74f-5d3e90e47efb'
client_secret = 'de5cf33d-fbee-4368-8fe9-e803490fe096'
client_id_pro = 'b52892d3-0711-4ccf-ab88-8572db0a4444'
client_secret_pro = '01ffb86e-5f4a-4a32-9b84-610dbd08aa3f'

In [276]:
def get_token_prod():
    token_url = 'https://oauth.piste.gouv.fr/api/oauth/token'  # URL de production
    token_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id_pro,
        'client_secret': client_secret_pro,
        'scope': 'openid'
    }
    response = requests.post(token_url, data=token_data)
    print(response.text)  # Affiche le contenu de la réponse pour diagnostic
    response.raise_for_status()  # Vérifie les erreurs
    token_info = response.json()
    access_token = token_info['access_token']
    return access_token

In [277]:
acess_token_prod = get_token_prod()

{"access_token":"br7R4JlPw5OnKyrgzd8MVPEoIcbbYfyeoiBv05HuolPvJmjW8JoSC2","token_type":"Bearer","expires_in":3600,"scope":"openid resource.READ"}


In [278]:
def get_token():
    token_url = 'https://sandbox-oauth.piste.gouv.fr/api/oauth/token'
    #inject cred
    token_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'openid'}
    response = requests.post(token_url, data=token_data)
    response.raise_for_status()  # vérif  erreurs
    # récup  jeton
    token_info = response.json()
    global access_token
    access_token = token_info['access_token']
    return access_token


In [279]:
access_token = get_token()

In [280]:
def ping_pong_test():
    headers_1 = {'accept': 'text/plain', 'Authorization': 'Bearer ' + access_token}
    output = requests.get("https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/ping", headers=headers_1).text
    display(output)

In [281]:
ping_pong_test()

'pong'

### Step 2: get dossier legislatif

Article modificateur d'une ordonnance a dossier législatif (via ordonnance):

In [282]:
def get_doss_legi(textId, date): # LEGITEXT000006075116 2021-04-15
  url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/legiPart'
  headers = {'accept': 'application/json','Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
  data = { #"searchedString":"constitution 1958",
        "date": date,
        "textId":textId }
  response = requests.post(url= url, headers =headers, json= data )
  return response.json()["dossiersLegislatifs"][0]

In [283]:
doss_legi = get_doss_legi("LEGIARTI000041568889", "2020-02-14")
print(doss_legi)

{'id': 'JORFDOLE000041568751', 'ancienId': None, 'origine': None, 'url': None, 'nature': None, 'titre': 'Ordonnance n° 2020-115 du 12 février 2020 renforçant le dispositif national de lutte contre le blanchiment de capitaux et le financement du terrorisme', 'type': None, 'dateCreation': None, 'dateDerniereModification': 1607040000000, 'legislature': None, 'echeancier': '', 'dateMajEcheancier': None, 'exposeMotif': None, 'arborescence': None, 'dossiers': [], 'refInjection': None, 'idTechInjection': None}


### Step 3: get list of directives

De dossier législatif aux numéros de directives

In [284]:
def get_directives_list(id): #JORFDOLE000038049286
	headers = {'accept': 'application/json','Content-Type': 'application/json','Authorization': 'Bearer ' + access_token}
	url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/dossierLegislatif'
	data = {"id":id}
	reponse = requests.post(headers = headers , url= url, json= data)
	return reponse.json()['dossierLegislatif']['dossiers']

In [285]:
get_directives_list("JORFDOLE000043697101")

[{'idTexte': 'JORFTEXT000043694571',
  'libelleTexte': "Ordonnance n° 2021-796 du 23 juin 2021 portant transposition de la directive (UE) 2019/2034 du Parlement européen et du Conseil du 27 novembre 2019 concernant la surveillance prudentielle des entreprises d'investissement",
  'contenuDossier': ''},
 {'idTexte': 'JORFTEXT000043694547',
  'libelleTexte': "Rapport au Président de la République relatif à l'ordonnance n° 2021-796 du 23 juin 2021 portant transposition de la directive (UE) 2019/2034 du Parlement européen et du Conseil du 27 novembre 2019 concernant la surveillance prudentielle des entreprises d'investissement",
  'contenuDossier': ''},
 {'idTexte': 'JORFTEXT000039454574',
  'libelleTexte': 'Directive (UE) 2019/2034 du Parlement européen et du Conseil du 27 novembre 2019 concernant la surveillance prudentielle des entreprises d’investissement et modifiant les directives 2002/87/CE, 2009/65/CE, 2011/61/UE, 2013/36/UE, 2014/59/UE et 2014/65/UE (Texte présentant de l’intérêt 

In [286]:
def get_directives_list_vprod(id):
    url = 'https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/dossierLegislatif'  # URL de production
    headers = {'accept': 'application/json','Content-Type': 'application/json','Authorization': 'Bearer ' + acess_token_prod  }
    data = {"id":id}
    response = requests.post(url=url, headers=headers, json=data)
    response_json = response.json()
    if response_json.get('status') == 'Internal Server Error':
        return "" 
    else:
        return response_json['dossierLegislatif']['dossiers']

In [287]:
liste  = get_directives_list("JORFDOLE000043697101")
liste

[{'idTexte': 'JORFTEXT000043694571',
  'libelleTexte': "Ordonnance n° 2021-796 du 23 juin 2021 portant transposition de la directive (UE) 2019/2034 du Parlement européen et du Conseil du 27 novembre 2019 concernant la surveillance prudentielle des entreprises d'investissement",
  'contenuDossier': ''},
 {'idTexte': 'JORFTEXT000043694547',
  'libelleTexte': "Rapport au Président de la République relatif à l'ordonnance n° 2021-796 du 23 juin 2021 portant transposition de la directive (UE) 2019/2034 du Parlement européen et du Conseil du 27 novembre 2019 concernant la surveillance prudentielle des entreprises d'investissement",
  'contenuDossier': ''},
 {'idTexte': 'JORFTEXT000039454574',
  'libelleTexte': 'Directive (UE) 2019/2034 du Parlement européen et du Conseil du 27 novembre 2019 concernant la surveillance prudentielle des entreprises d’investissement et modifiant les directives 2002/87/CE, 2009/65/CE, 2011/61/UE, 2013/36/UE, 2014/59/UE et 2014/65/UE (Texte présentant de l’intérêt 

In [288]:
# Extraction des idTexte
id_textes = [item['idTexte'] for item in liste]
print(id_textes)

['JORFTEXT000043694571', 'JORFTEXT000043694547', 'JORFTEXT000039454574']


In [289]:
def get_directives_list_v2(textId):
    url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/legiPart'
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    data = { "textId": textId}
    try:
        response = requests.post(url=url, headers=headers, json=data)
        response.raise_for_status()  # Vérifie les erreurs HTTP
        liste = response.json()['dossierLegislatif']['dossiers']
        #id_textes = [item['idTexte'] for item in liste]
        return response
        #return response.json()["dossiersLegislatifs"][0]['id'] 
    except requests.RequestException as e:
        print(f"Erreur de requête : {e} pour textId={textId} .")
        return None
    except KeyError:
        print(f"L'identifiant n'a pas pu être récupéré pour textId={textId} .")
        return None

In [290]:
get_directives_list_v2("JORFDOLE000041568751")

Erreur de requête : 500 Server Error: Internal Server Error for url: https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/legiPart pour textId=JORFDOLE000041568751 .


In [291]:
def get_directives_list_vprod(id):
    url = 'https://api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/dossierLegislatif'  # URL de production
    headers = {'accept': 'application/json','Content-Type': 'application/json','Authorization': 'Bearer ' + acess_token_prod  }
    data = {"id":id}
    response = requests.post(url=url, headers=headers, json=data)
    return response.json()['dossierLegislatif']['dossiers']

def get_directives_list(id): #JORFDOLE000038049286
	headers = {'accept': 'application/json','Content-Type': 'application/json','Authorization': 'Bearer ' + access_token}
	url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/dossierLegislatif'
	data = {"id":id}
	reponse = requests.post(headers = headers , url= url, json= data)
	return reponse.json()['dossierLegislatif']['dossiers']


In [292]:
get_directives_list_vprod("JORFDOLE000043697101")

[{'idTexte': 'JORFTEXT000043694571',
  'libelleTexte': "Ordonnance n° 2021-796 du 23 juin 2021 portant transposition de la directive (UE) 2019/2034 du Parlement européen et du Conseil du 27 novembre 2019 concernant la surveillance prudentielle des entreprises d'investissement",
  'contenuDossier': ''},
 {'idTexte': 'JORFTEXT000043694547',
  'libelleTexte': "Rapport au Président de la République relatif à l'ordonnance n° 2021-796 du 23 juin 2021 portant transposition de la directive (UE) 2019/2034 du Parlement européen et du Conseil du 27 novembre 2019 concernant la surveillance prudentielle des entreprises d'investissement",
  'contenuDossier': ''},
 {'idTexte': 'JORFTEXT000039454574',
  'libelleTexte': 'Directive (UE) 2019/2034 du Parlement européen et du Conseil du 27 novembre 2019 concernant la surveillance prudentielle des entreprises d’investissement et modifiant les directives 2002/87/CE, 2009/65/CE, 2011/61/UE, 2013/36/UE, 2014/59/UE et 2014/65/UE (Texte présentant de l’intérêt 

### Step 4: get Celex of directive from JORF

De numéros de directives a Celex de directive

In [293]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [294]:
def get_celex_full(textCid :str): #JORFTEXT000042636234
  url= 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/jorf'
  headers= {'accept': 'application/json',
		'Content-Type': 'application/json',
		'Authorization': 'Bearer ' + access_token}
  data= {"searchedString":"","textCid":textCid}
  response = requests.post( headers =headers, url= url, json= data )
  response_json = response.json()
  return response_json

In [295]:
get_celex_full("JORFTEXT000043694571")

{'executionTime': 5,
 'dereferenced': False,
 'id': 'JORFTEXT000043694571_01-01-2999',
 'idConteneur': None,
 'cid': 'JORFTEXT000043694571',
 'title': "Ordonnance n° 2021-796 du 23 juin 2021 portant transposition de la directive (UE) 2019/2034 du Parlement européen et du Conseil du 27 novembre 2019 concernant la surveillance prudentielle des entreprises d'investissement",
 'nor': 'ECOT2113069R',
 'eli': '/eli/ordonnance/2021/6/23/ECOT2113069R/jo/texte',
 'alias': '/eli/ordonnance/2021/6/23/2021-796/jo/texte',
 'jorfText': 'JORF n°0145 du 24 juin 2021',
 'jurisState': 'Sans état juridique',
 'visa': '<p align="left"><br/>Le Président de la République,<br/>Sur le rapport du Premier ministre et du ministre de l\'économie, des finances et de la relance,<br/>Vu la Constitution, notamment son article 38 ;<br/>Vu le règlement (UE) n° 1093/2010 du Parlement européen et du Conseil du 24 novembre 2010 instituant une Autorité européenne de surveillance (Autorité bancaire européenne), modifiant la

In [296]:

def get_celex(textCid :str): #JORFTEXT000042636234
  url= 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/jorf'
  headers= {'accept': 'application/json',
		'Content-Type': 'application/json',
		'Authorization': 'Bearer ' + access_token}
  data= {"searchedString":"","textCid":textCid}
  response = requests.post( headers =headers, url= url, json= data )
  response_json = response.json()
  return response_json

In [297]:
celex= get_celex("JORFTEXT000037171436") #AML JORFTEXT000037171436
print(celex)

{'executionTime': 2, 'dereferenced': False, 'id': 'JORFTEXT000037171436_01-01-2999', 'idConteneur': None, 'cid': 'JORFTEXT000037171436', 'title': "Directive (UE) 2018/843 du Parlement européen et du Conseil du 30 mai 2018 modifiant la directive (UE) 2015/849 relative à la prévention de l’utilisation du système financier aux fins du blanchiment de capitaux ou du financement du terrorisme ainsi que les directives 2009/138/CE et 2013/36/UE (Texte présentant de l'intérêt pour l'EEE)\n", 'nor': '32018L0843', 'eli': None, 'alias': None, 'jorfText': 'JOUE L 156 du 19 juin  2018', 'jurisState': 'Sans état juridique', 'visa': '', 'modifDate': None, 'jurisDate': None, 'dateDebutVersion': '2999-01-01', 'dateFinVersion': '2999-01-01', 'signers': '', 'prepWork': '', 'dateParution': 1529366400000, 'dateTexte': 1527638400000, 'numParution': None, 'notice': '', 'nota': '', 'inap': False, 'textNumber': None, 'textAbroge': False, 'etat': None, 'dossiersLegislatifs': [{'id': 'JORFDOLE000041568751', 'anci

### Step 5: get text from Eurlex with Sparql in Cellar database 

In [298]:
def celextourl_data(endpoint_url, celex_id):

    # Préparer la requête SPARQL
    query = f"""
    PREFIX cdm: <http://publications.europa.eu/ontology/cdm#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT ?property ?value
    WHERE {{
      ?acte cdm:resource_legal_id_celex "{celex_id}"^^xsd:string ;
            ?property ?value .
    }}
    """

    # Initialiser le wrapper SPARQL
    sparql = SPARQLWrapper(endpoint_url) #inject url endpoint
    sparql.setQuery(query)               # inject query
    sparql.setReturnFormat(JSON)          # set in json format
    results = sparql.query().convert()   # run
    return results

In [299]:
json_res= celextourl_data('https://publications.europa.eu/webapi/rdf/sparql', celex)

URITooLong: URITooLong: The URI requested by the client is longer than the server is willing to interpret. Check if the request was sent using GET method instead of POST method.. 

Response:
b'<html>\r\n<head><title>414 Request-URI Too Large</title></head>\r\n<body>\r\n<center><h1>414 Request-URI Too Large</h1></center>\r\n</body>\r\n</html>\r\n'

In [54]:
def extract_celex_link(data):
    return next(
        (item['value']['value'] for item in data['results']['bindings']
         if 'value' in item and 'value' in item['value'] and 'http://publications.europa.eu/resource/celex/' in item['value']['value']),
        None
    )

In [55]:
directive_lien= extract_celex_link(json_res)
print(directive_lien)

None


### Step 6: Firecrawl (option 1)

In [56]:
from firecrawl import FirecrawlApp

app = FirecrawlApp(api_key="fc-1b2bc67fc7734320ba9135983d78d4e3")

# Scrape a website:
scrape_result = app.scrape_url(directive_lien, params={'formats': ['markdown']})
# scrape_result = app.scrape_url(directive_lien, params={'formats': ['markdown', 'html']})

print(scrape_result)


HTTPError: Unexpected error during scrape URL: Status code 400. Bad Request - [{'code': 'custom', 'message': 'URL must have a valid top-level domain or be a valid path', 'path': ['url']}]

In [25]:
display(scrape_result)


{'markdown': '|     |     |     |     |\n| --- | --- | --- | --- |\n| 19.6.2018 | EN | Official Journal of the European Union | L 156/43 |\n\n* * *\n\nDIRECTIVE (EU) 2018/843 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\n\n\nof 30 May 2018\n\n\namending Directive (EU) 2015/849 on the prevention of the use of the financial system for the purposes of money laundering or terrorist financing, and amending Directives 2009/138/EC and 2013/36/EU\n\n(Text with EEA relevance)\n\nTHE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION,\n\nHaving regard to the Treaty on the Functioning of the European Union, and in particular Article 114 thereof,\n\nHaving regard to the proposal from the European Commission,\n\nAfter transmission of the draft legislative act to the national parliaments,\n\nHaving regard to the opinion of the European Central Bank\xa0[(1)](https://publications.europa.eu/resource/cellar/6ea73588-7380-11e8-9483-01aa75ed71a1.0006.03/DOC_1#ntr1-L_2018156EN.01004301-E0001),\n\nH

In [26]:
# Exportation des données vers le fichier JSON
with open("json_export_directive_inMarkdown", 'w', encoding='utf-8') as fichier:
    json.dump(scrape_result, fichier, indent=4, ensure_ascii=False)
